In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\2101\\OneDrive\\Desktop\\git\\Brain-Tumor-MRI-Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    checkpoint_model_filepath: Path
    tensorboard_root_log_dir: Path

In [4]:
from brainTumorMRIClassification.constants import *
from brainTumorMRIClassification.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])
        
        return PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir)
        )

In [5]:
# Component
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [6]:

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    @property
    def _create_early_stopping(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            min_delta=1e-9,
            patience=8,
            verbose=True
        )

    @property
    def _create_reduce_lr(self):
        return tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.3,
            patience=5,
            verbose=True
        )

    def get_all_callbacks(self):
        return [
            self._create_early_stopping,
            self._create_reduce_lr
        ]


In [7]:
# Pipeline
try:
    config=ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_all_callbacks()
except Exception as e:
    raise e


[2025-09-03 12:26:16,189] : INFO : common : YAML file config\config.yaml loaded successfully.
[2025-09-03 12:26:16,192] : INFO : common : YAML file params.yaml loaded successfully.
[2025-09-03 12:26:16,195] : INFO : common : Created directory: artifacts
[2025-09-03 12:26:16,199] : INFO : common : Created directory: artifacts\prepare_callbacks\checkpoint_dir
[2025-09-03 12:26:16,202] : INFO : common : Created directory: artifacts\prepare_callbacks\tensorboard_log_dir
